In [4]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, TimestampType

# Define schema for flattened research paper data with additional fields
schema = StructType(
    [
        StructField('hash_id', LongType(), True),  # Unique hash ID for the paper
        StructField('title', StringType(), True),  # Title of the paper
        StructField('authors', StringType(), True),  # Authors of the paper as a comma-separated string
        StructField('published', TimestampType(), True),  # Publication date of the paper
        StructField('summary', StringType(), True),  # Abstract or summary of the paper
        StructField('pdf_url', StringType(), True),  # URL to download the paper's PDF
        StructField('entry_id', StringType(), True),  # Unique entry ID for the paper (e.g., arXiv ID)
        StructField('recommended', LongType(), True),  # Flag indicating if the paper is recommended (1 if recommended, 0 otherwise)
        StructField('referenceCount', LongType(), True),  # Number of references in the paper
        StructField('citationCount', LongType(), True),  # Number of times the paper has been cited
        StructField('references', StringType(), True),  # JSON string of references
        StructField('citations', StringType(), True),  # JSON string of citations
        StructField('s2FieldsOfStudy', StringType(), True),  # JSON string of fields of study
        StructField('tldr', StringType(), True),  # JSON string for TLDR summary
        StructField('query_id', StringType(), True),  # Identifier for the query that generated the result
        StructField('Tag_1', StringType(), True),  # Additional tag 1
        StructField('Tag_2', StringType(), True),  # Additional tag 2
        StructField('Tag_3', StringType(), True),  # Additional tag 3
        StructField('Tag_4', StringType(), True),  # Additional tag 4
        StructField('Tag_5', StringType(), True),  # Additional tag 5
        StructField('field', StringType(), True)  # Field of the paper
    ]
)

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 8, Finished, Available, Finished)

In [5]:
# pipeline parameter
filename = ""


StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 9, Finished, Available, Finished)

In [6]:
# while filename.split(".")[-1] not in list(spark.catalog.listTables("bronze")):
#     pass

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 10, Finished, Available, Finished)

In [7]:
df = spark.sql(f"SELECT * FROM {filename}")
df_pd = df.toPandas()

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 11, Finished, Available, Finished)

In [9]:
import pandas as pd
from openai import OpenAI
def add_hierarchical_tags_to_df(df, summary_column='summary', num_tags=5, api_key=None):
    """
    Adds hierarchical tags to the DataFrame based on the summaries, ensuring consistency with existing tags.

    Parameters:
    - df: pandas DataFrame containing a summary column.
    - summary_column: name of the summary column in df.
    - num_tags: total number of tags desired (existing + new).
    - api_key: OpenAI API key. If None, will use the OPENAI_API_KEY environment variable.

    Returns:
    - df_with_tags: DataFrame with new tag columns added.
    """
    # Set the OpenAI API key
    if not api_key:
        print("no api.")
        return

    # Check which tags already exist in the DataFrame
    existing_columns = set(df.columns)
    required_tag_columns = {f'Tag_{i}' for i in range(1, num_tags + 1)}
    missing_tag_columns = sorted(list(required_tag_columns - existing_columns), key=lambda x: int(x.split('_')[1]))

    # If all required tags are already in the DataFrame, return the original DataFrame
    if not missing_tag_columns:
        print("All required tags already exist in the DataFrame.")
        return df

    def generate_hierarchical_tags(row):
        summary_text = row[summary_column]

        # Extract existing tags for this row
        existing_tags = []
        for i in range(1, num_tags + 1):
            tag_column = f'Tag_{i}'
            if tag_column in existing_columns and pd.notnull(row.get(tag_column, None)):
                existing_tags.append(row[tag_column])

        # Prepare the messages for the OpenAI API
        existing_tags_text = ""
        start_tag_number = 1
        if existing_tags:
            existing_tags_text = "Existing Tags:\n" + '\n'.join([f"{i}. {tag}" for i, tag in enumerate(existing_tags, 1)])
            start_tag_number = len(existing_tags) + 1

        num_new_tags = num_tags - len(existing_tags)
        if num_new_tags <= 0:
            # All tags are already present for this row
            return {}

        messages = [
            {
                "role": "system",
                "content": "You are an AI assistant that continues hierarchical tags from general to specific based on the given summary and existing tags."
            },
            {
                "role": "user",
                "content": f"""Continue the hierarchical list of tags for the following summary, starting from tag number {start_tag_number}, and generate {num_new_tags} additional tags to reach a total of {num_tags} tags.

{existing_tags_text}

Summary:
{summary_text}

New Tags:
""" + '\n'.join([f"{i}." for i in range(start_tag_number, num_tags + 1)])
            },
        ]
        # Call the OpenAI ChatCompletion API
        client = OpenAI(api_key=api_key)
        response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=num_new_tags*20,
        temperature=0,
        n=1,
        stop=None
    )

        # Extract tags from the response
        tags_text = response.choices[0].message.content

        # Parse the tags into a dictionary
        tags = {}
        for line in tags_text.strip().split('\n'):
            line = line.strip()
            if line and '.' in line:
                parts = line.split('.', 1)
                if parts[0].isdigit():
                    tag_number = int(parts[0])
                    tag_column = f'Tag_{tag_number}'
                    if tag_column in missing_tag_columns:
                        tag_text = parts[1].strip()
                        tags[tag_column] = tag_text
        return tags


    # Apply the function to each row
    tags_series = df.apply(generate_hierarchical_tags, axis=1)
    tags_df = pd.DataFrame(tags_series.tolist())

    # Concatenate the original DataFrame with the new tags DataFrame
    df_with_tags = pd.concat([df.reset_index(drop=True), tags_df.reset_index(drop=True)], axis=1)

    return df_with_tags

api = ""
df_pd_withTags = add_hierarchical_tags_to_df(df_pd,num_tags=5,summary_column="summary",api_key=api)

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 13, Finished, Available, Finished)

In [10]:
def saveAsTable(source, name):
    df = pd.DataFrame(source)
    spark_df = spark.createDataFrame(df,schema=schema)
    spark_df.write.mode("append").format("delta").saveAsTable(name)


def TsaveAsTable(source, name):
    df = pd.DataFrame(source)
    spark_df = spark.createDataFrame(df,schema=schema)
    spark_df.write.mode("overwrite").format("delta").saveAsTable(name)

def appendTable(source,name):
    df = pd.DataFrame(source)
    spark_df = spark.createDataFrame(df,schema=schema)
    spark_df.write.mode("append").format("delta").saveAsTable(name)

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 14, Finished, Available, Finished)

In [13]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 17, Finished, Available, Finished)

In [17]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from collections import Counter

# Flatten all tag columns into a single list and remove NaN values
all_tags_flat = []

tag_columns = [f"Tag_{i}" for i in range(1,6)]

for col in tag_columns:
    all_tags_flat.extend(df_pd_withTags[col].dropna().tolist())

# Count the occurrences of each tag
tag_counts = Counter(all_tags_flat)

# Convert to a DataFrame for easier readability
tag_counts_df = pd.DataFrame(tag_counts.items(), columns=['Tag', 'Count']).sort_values(by='Count', ascending=False)


StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 29, Finished, Available, Finished)

In [9]:
# Step 2: Compute Tag Embeddings

# Load the pre-trained model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all tags
# tag_embeddings = model.encode(all_tags, show_progress_bar=True)

# Step 3: Perform Hierarchical Clustering

# Perform hierarchical clustering using Ward's method
# linked = linkage(tag_embeddings, method='ward')

# Step 4: Visualize the Tag Hierarchy

# Set up the plot size
# plt.figure(figsize=(12, 20))

# # Plot the dendrogram
# dendrogram(
#     linked,
#     orientation='left',
#     labels=all_tags,
#     distance_sort='descending',
#     show_leaf_counts=True,
#     leaf_font_size=10
# )

# plt.title('Hierarchical Clustering Dendrogram for Tags')
# plt.xlabel('Distance')
# plt.ylabel('Tags')
# plt.tight_layout()
# plt.show()

StatementMeta(, cf401b79-980e-4a4a-a9d4-90743c633403, 13, Finished, Available, Finished)

In [16]:
# from scipy.cluster.hierarchy import fcluster

# # Define the maximum distance threshold
# max_d = 4  # Adjust based on your dendrogram

# # Get cluster labels
# cluster_labels = fcluster(linked, max_d, criterion='distance')

# # Create a DataFrame of tags and their cluster labels
# tag_clusters = pd.DataFrame({'Tag': all_tags, 'Cluster': cluster_labels})
# c=0
# # Display the clusters
# for cluster_num in sorted(tag_clusters['Cluster'].unique()):
#     cluster_tags = tag_clusters[tag_clusters['Cluster'] == cluster_num]['Tag'].tolist()
#     # print(f"\nCluster {cluster_num}:")
#     c+=1
#     # for tag in cluster_tags:
#         # print(f" - {tag}")
#         # c+=1

StatementMeta(, 3ced3996-9d59-4c40-9cf3-96f1662c55f8, 21, Finished, Available, Finished)

,Tag,Cluster
0,Nonlinear operations,56
1,Simulation-Based Inference,12
2,Human behavior analysis,108
3,Gender Bias,110
4,Robustness Testing,23
...,...,...
9437,Data Filtering,119
9438,Quantum Phases,65
9439,Model-Free Approach,8
9440,Error-Correction Strategies,25


In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import pandas as pd

all_tags = all_tags_flat

# Step 1: Load the pre-trained model and compute embeddings for all tags
model = SentenceTransformer('all-MiniLM-L6-v2')
tag_embeddings = model.encode(all_tags, show_progress_bar=True)

# Step 2: Define the 10 specific tags to serve as cluster centers
predefined_tags = list(tag_counts_df[:10]["Tag"])
# Compute embeddings for these predefined tags
predefined_embeddings = model.encode(predefined_tags)

# Step 3: Run K-means clustering with predefined initial centers
kmeans = KMeans(n_clusters=min(10,len(tag_counts_df)), init=predefined_embeddings, n_init=1, random_state=0)
kmeans.fit(tag_embeddings)
cluster_labels = kmeans.labels_

# Step 4: Organize Tags by Cluster
# Create a DataFrame with tags and their assigned clusters
tag_clusters = pd.DataFrame({'Tag': all_tags, 'Cluster': cluster_labels})

# Group tags by their cluster labels
grouped_tags = tag_clusters.groupby('Cluster')['Tag'].apply(list).reset_index()

# Display the grouped tags
print(grouped_tags)

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 30, Finished, Available, Finished)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   Cluster                           Tag
0        0          [Robotics, Robotics]
1        1         [Multi-robot systems]
2        2           [Autonomous Robots]
3        3      [Decentralized planning]
4        4           [Search Algorithms]
5        5  [Graph Neural Network (GNN)]
6        6              [Ergodic Search]
7        7      [Scalability challenges]
8        8          [Obstacle Avoidance]


StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 31, Finished, Available, Finished)

In [21]:
grouped_tags["Cluster"] = grouped_tags["Cluster"].apply(lambda x: predefined_tags[x])
# grouped_tags

StatementMeta(, f0665b10-b9d8-4a64-9f35-fa819a5f1748, 34, Finished, Available, Finished)

,Cluster,Tag
0,Robotics,"[Robotics, Robotics]"
1,Multi-robot systems,[Multi-robot systems]
2,Autonomous Robots,[Autonomous Robots]
3,Decentralized planning,[Decentralized planning]
4,Search Algorithms,[Search Algorithms]
5,Graph Neural Network (GNN),[Graph Neural Network (GNN)]
6,Ergodic Search,[Ergodic Search]
7,Scalability challenges,[Scalability challenges]
8,Obstacle Avoidance,[Obstacle Avoidance]


In [ ]:
# Step 1: Create a dictionary mapping tags to predefined clusters
tag_to_cluster = {}
for _, row in grouped_tags.iterrows():
    cluster_name = row["Cluster"]
    for tag in row["Tag"]:
        tag_to_cluster[tag] = cluster_name


# Step 2: Define a function to assign clusters to each entry
def assign_cluster(row):
    tags = [row['Tag_1'], row['Tag_2'], row['Tag_3'], row['Tag_4'], row['Tag_5']]
    
    # First, check for direct match
    for tag in tags:
        if tag in predefined_tags:
            return tag  # Return the cluster name directly if there's an exact match
    
    # If no direct match, proceed with majority voting
    cluster_votes = {}
    for tag in tags:
        cluster = tag_to_cluster.get(tag)
        if cluster:
            if cluster not in cluster_votes:
                cluster_votes[cluster] = 0
            cluster_votes[cluster] += 1
    
    # Assign the cluster with the most votes
    if cluster_votes:
        return max(cluster_votes, key=cluster_votes.get)
    
    return None  # Return None if no cluster could be assigned

# Step 3: Apply the function to each row in df_pd_withTags to assign clusters
df_pd_withTags['field'] = df_pd_withTags.apply(assign_cluster, axis=1)

StatementMeta(, 3c7c89e9-dbbd-42ce-9f78-1aa3cec9d311, 18, Finished, Available, Finished)

,hash_id,title,authors,published,summary,pdf_url,entry_id,recommended,referenceCount,citationCount,...,citations,s2FieldsOfStudy,tldr,query_id,Tag_1,Tag_2,Tag_3,Tag_4,Tag_5,field
0,918220639,Data is missing again -- Reconstruction of pow...,"Amandine Pierrot, Pierre Pinson",2024-08-30 23:58:28,The risk of missing data and subsequent incomp...,http://arxiv.org/pdf/2409.00300v1,http://arxiv.org/abs/2409.00300v1,0,31,0,...,,Computer Science|Mathematics|Engineering|Envir...,None,456,Data Imputation,Wind Farm Monitoring,Spectral Graph Theory,Nearest Neighbor Imputation,Offshore Wind Farms,Machine Learning
1,1939709920,On Expressive Power of Quantized Neural Networ...,"Geonho Hwang, Yeachan Park, Sejun Park",2024-08-30 23:40:19,Research into the expressive power of neural n...,http://arxiv.org/pdf/2409.00297v1,http://arxiv.org/abs/2409.00297v1,0,0,0,...,,Computer Science|Mathematics|Computer Science|...,This work provides a necessary and sufficient ...,456,Neural Networks,Quantized Networks,Fixed-Point Arithmetic,Activation Functions,Universal Approximation,Neural Networks
2,1918891479,Credit Scores: Performance and Equity,"Stefania Albanesi, Domonkos F. Vamossy",2024-08-30 23:36:02,Credit scores are critical for allocating cons...,http://arxiv.org/pdf/2409.00296v1,http://arxiv.org/abs/2409.00296v1,0,0,0,...,,Computer Science|Economics|Economics|Business|...,This work benchmarks a widely used credit scor...,456,Financial Services,Credit Scoring,Consumer Debt,Machine Learning Models,Equitable Access to Credit,Neural Networks
3,1880705934,Box2Flow: Instance-based Action Flow Graphs fr...,"Jiatong Li, Kalliopi Basioti, Vladimir Pavlovic",2024-08-30 23:33:19,A large amount of procedural videos on the web...,http://arxiv.org/pdf/2409.00295v1,http://arxiv.org/abs/2409.00295v1,0,41,0,...,,Computer Science|Computer Science|,"Box2Flow is proposed, an instance-based method...",456,Procedural videos,Task completion,Flow graphs,Box2Flow method,Step relationships,Artificial Intelligence
4,1910186425,Quantum Machine Learning for Anomaly Detection...,"Sounak Bhowmik, Himanshu Thapliyal",2024-08-30 23:28:00,Anomaly detection is a crucial task in cyber s...,http://arxiv.org/pdf/2409.00294v1,http://arxiv.org/abs/2409.00294v1,0,23,0,...,,Computer Science|Physics|Computer Science|Engi...,A generic framework for applying QML algorithm...,456,Cyber Security,Anomaly Detection,Quantum Machine Learning,Consumer Electronics,Machine Learning Algorithms,Machine Learning
5,1378396479,Benchmarking the Performance of Large Language...,"Zuoning Zhang, Dhruv Parikh, Youning Zhang, Vi...",2024-08-30 22:45:49,Transformer based Large Language Models (LLMs)...,http://arxiv.org/pdf/2409.00287v2,http://arxiv.org/abs/2409.00287v2,0,27,0,...,,Computer Science|Computer Science|,This paper benchmarks the effectiveness of thi...,456,Artificial Intelligence,Machine Learning,Large Language Models,Hardware Acceleration,Performance Optimization,Artificial Intelligence
6,1349392230,Reframing Data Value for Large Language Models...,"Mohamad Rida Rammal, Ruida Zhou, Suhas Diggavi",2024-08-30 22:32:24,Data valuation seeks to answer the important q...,http://arxiv.org/pdf/2409.00284v2,http://arxiv.org/abs/2409.00284v2,0,23,0,...,,Computer Science|Computer Science|Linguistics|,This work proposes an alternative perspective ...,456,Data Valuation,Data Worth,Language Models,Value Function,Plausibility Assessment,Language Models
7,1175210199,Boosting K-means for Big Data by Fusing Data S...,"Ravil Mussabayev, Rustam Mussabayev",2024-10-18 15:43:34,K-means clustering is a cornerstone of data mi...,http://arxiv.org/pdf/2410.14548v1,http://arxiv.org/abs/2410.14548v1,1,15,0,...,,Computer Science|Mathematics|Computer Science|...,A novel heuristic algorithm that leverages the...,456,Data Mining,Clustering Algorithms,K-means Clustering,Metaheuristic Optimization,Big Data Optimization,Machine Learning
8,2292658129,K-Means Clustering With Incomplete Data with t...,"Lovis Kwasi Armah, Igor Melnykov"

In [17]:
df_pd_withTags.info()

StatementMeta(, 3c7c89e9-dbbd-42ce-9f78-1aa3cec9d311, 24, Finished, Available, Finished)

KeyError: 'There are no fields in dtype int64.'

In [20]:

f_name = filename.split('.')[-1].split('_')
query = f_name[2]
query_id = f_name[1]
silver_filename = f"silver.{'_'+query_id+'_'+query}"
saveAsTable(df_pd_withTags,f"{silver_filename}_tagged")

StatementMeta(, 3c7c89e9-dbbd-42ce-9f78-1aa3cec9d311, 27, Finished, Available, Finished)

In [25]:
# import pandas as pd
# import numpy as np


# df = spark.sql("SELECT * FROM Gold.target")

# df_pd = df.toPandas()

# # Assuming df_pd is your DataFrame
# num_rows = len(df_pd)
# # Randomly generate an array with half `1`s and half `2`s
# ids = np.random.choice([1, 2], size=num_rows, p=[0.5, 0.5])

# # Ensure exactly half are `1` and half are `2`
# # Adjust counts in case of any imbalance due to randomness
# ids[:num_rows // 2] = 1
# ids[num_rows // 2:] = 2
# np.random.shuffle(ids)  # Shuffle to ensure randomness in assignment

# # Add the `id` column to df_pd
# # df_pd['id'] = ids

# df_pd = df_pd.dropna()

# TsaveAsTable(df_pd,"Gold.Target")

# df_pd["hash_id"]= 123
# saveAsTable(df_pd,"Gold.Target")


StatementMeta(, 68e0114a-a28f-46ea-bc85-0917a89d573e, 29, Finished, Available, Finished)

In [26]:
# df_pd

StatementMeta(, 68e0114a-a28f-46ea-bc85-0917a89d573e, 30, Finished, Available, Finished)

,hash_id,title,authors,published,summary,pdf_url,entry_id,Tag_1,Tag_2,Tag_3,Tag_4,Tag_5,field,id
0,2136994751,EMMA: End-to-End Multimodal Model for Autonomo...,"Jyh-Jing Hwang, Runsheng Xu, Hubert Lin, Wei-C...",2024-10-30,"We introduce EMMA, an End-to-end Multimodal Mo...",http://arxiv.org/pdf/2410.23262v1,http://arxiv.org/abs/2410.23262v1,Autonomous driving,Multimodal model,End-to-end model,Large language model,Motion planning,Language Models,2
1,3744028288,Reliability of Topic Modeling,"Kayla Schroeder, Zach Wood-Doughty",2024-10-30,Topic models allow researchers to extract late...,http://arxiv.org/pdf/2410.23186v1,http://arxiv.org/abs/2410.23186v1,Topic Modeling,Latent Factors,Statistical Analysis,Model Reliability,Measurement Theory,Language Models,2
2,2962536164,Provably Optimal Memory Capacity for Modern Ho...,"Jerry Yao-Chieh Hu, Dennis Wu, Han Liu",2024-10-30,We study the optimal memorization capacity of ...,http://arxiv.org/pdf/2410.23126v1,http://arxiv.org/abs/2410.23126v1,Memory Capacity Analysis,Hopfield Models,Kernelized Hopfield Models,Spherical Codes,Memory Retrieval Algorithms,Language Models,1
3,3919570361,Consistency Diffusion Bridge Models,"Guande He, Kaiwen Zheng, Jianfei Chen, Fan Bao...",2024-10-30,Diffusion models (DMs) have become the dominan...,http://arxiv.org/pdf/2410.22637v2,http://arxiv.org/abs/2410.22637v2,Generative Modeling,Diffusion Models,Diffusion Denoising Bridge Models (DDBMs),Consistency Models,Probability-Flow Ordinary Differential Equatio...,Language Models,2
4,2320061627,Lévy graphical models,"Sebastian Engelke, Jevgenijs Ivanovs, Jakob D....",2024-10-25,Conditional independence and graphical models ...,http://arxiv.org/pdf/2410.19952v1,http://arxiv.org/abs/2410.19952v1,Graphical Models,Conditional Independence,L\'evy Processes,Statistical Modeling,Stochastic Processes,Language Models,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,2641228975,A Simple Model of Inference Scaling Laws,Noam Levi,2024-10-21,Neural scaling laws have garnered significant ...,http://arxiv.org/pdf/2410.16377v1,http://arxiv.org/abs/2410.16377v1,Machine Learning,Neural Networks,Inference,Scaling Laws,Generative Models,Machine Learning,1
3946,2295460765,Hydra-LSTM: A semi-shared Machine Learning arc...,"Karan Ruparell, Robert J. Marks, Andy Wood, Ki...",2024-10-21,Long Short Term Memory networks (LSTMs) are us...,http://arxiv.org/pdf/2410.16343v1,http://arxiv.org/abs/2410.16343v1,Machine Learning,Neural Networks,River Discharge Prediction,Hydra-LSTM,Catchment-specific Data Integration,Machine Learning,1
3947,4105279788,On the VC dimension of deep group convolutiona...,"Anna Sepliarskaia, Sophie Langer, Johannes Sch...",2024-10-21,We study the generalization capabilities of Gr...,http://arxiv.org/pdf/2410.15800v1,http://arxiv.org/abs/2410.15800v1,Machine Learning,Neural Networks,Convolutional Neural Networks (CNNs),Group Convolutional Neural Networks (GCNNs),Vapnik-Chervonenkis (VC) Dimension,Machine Learning,2
3948,1741714813,ViMoE: An Empirical Study of Designing Vision ...,"Xumeng Han, Longhui Wei, Zhiyang Dou, Zipeng W...",2024-10-21,Mixture-of-Experts (MoE) models embody the div...,http://arxiv.org/pdf/2410.15732v1,http://arxiv.org/abs/2410.15732v1,Machine Learning,Neural Networks,Vision Transformer,Image Classification,Expert Models,Machine Learning,1
